In [2]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt

from IPython import display

POP_SIZE = 100
CROSS_RATE = 0.08
MUTATION_RATE = 0.03
N_GENERATIONS = 200
X_BOUND = [-10,10]
Y_BOUND = [-200,100]

def getDPA(num):
    # get_Decimal_Point_Accuracy
    i = 0
    while num < 1:
        num *= 10
        i += 1
    return i

ALPHA = 0.1
ROUND_KEEP = getDPA(ALPHA)

X_BOUND_SIZE = abs(X_BOUND[1])+abs(X_BOUND[0])
DNA_SIZE = math.ceil(np.log2(X_BOUND_SIZE/ALPHA))

DecDic = {}

def get_key (dict, value):
    for Dict_Key,Dict_Value in dict.items():
        if (Dict_Value == value).all():
            return Dict_Key
    return Dict_Key

def F(x): 
    return np.sin(10*x)*x + np.cos(2*x)*x

def get_fitness(pred): 
    return pred + 1e-3 - np.min(pred)

def initiDNA():
    global DecDic
    for i in range(0,int(X_BOUND_SIZE/ALPHA) + 1):
        decNdarray = np.arange(X_BOUND[0] ,X_BOUND[1]+1 ,ALPHA)
        target = (bin(((1 << DNA_SIZE) - 1) & i)[2:]).zfill(DNA_SIZE)
        line = np.array(list(map(int, target)))
        DecDic[round(decNdarray[i],ROUND_KEEP)] = line

    pop = np.zeros(POP_SIZE)
    for i in range(POP_SIZE):
        pop[i] = random.randrange(X_BOUND[0],X_BOUND[1]) + round(random.random(),ROUND_KEEP)
    return pop

def translateDNA(_pop):
    if type(_pop[0]) == np.ndarray: #to dec
        pop = np.zeros(POP_SIZE)
        #print("bin input")
        for index,Bin_Value in enumerate(_pop):
            pop[index] = get_key(DecDic,Bin_Value)
        return np.array(pop)
    
    if type(_pop[0]) == np.float64: #to bin
        pop = []
        #print("dec input")
        for index,Dec_Key in enumerate(_pop):
            pop.append(DecDic.get(round(Dec_Key,ROUND_KEEP)))
        return np.array(pop)

def select(pop, fitness):
    idx = np.random.choice(np.arange(len(pop)), size=POP_SIZE, replace=True,p=fitness/fitness.sum())
    return pop[idx]


def crossover(parent, pop):     # mating process (genes crossover)
    if np.random.rand() < CROSS_RATE:
        i_ = np.random.randint(0, POP_SIZE, size=1)                          # select another individual from pop
        cross_points = np.random.randint(0, 2, size=DNA_SIZE).astype(np.bool)   # choose crossover points
        parent[cross_points] = pop[i_, cross_points]                            # mating and produce one child
    return parent


def mutate(child):
    for point in range(DNA_SIZE):
        if np.random.rand() < MUTATION_RATE:
            child[point] = 1 if child[point] == 0 else 0
    return child


pop = initiDNA()
pop = translateDNA(pop)

plt.ion()

x = np.linspace(X_BOUND[0],X_BOUND[1],200)
plt.plot(x, F(x))

for _ in range(N_GENERATIONS):
    F_values = F(translateDNA(pop))
    #print(len(F_values))
    
    plt.scatter(translateDNA(pop), F_values, s=100, lw=0, c='red',alpha=ALPHA)
    plt.plot(x,F(x),linestyle="solid",color="black")
    plt.xlim = X_BOUND
    plt.ylim = Y_BOUND
    
    plt.rcParams['figure.figsize'] = (8.0,5.0)
    plt.rcParams['savefig.dpi'] = 100
    plt.rcParams['figure.dpi'] = 100
    
    plt.ioff(); plt.show()
    display.clear_output(wait=True)
    
    fitness = get_fitness(F_values)
    #print("Most fitted DNA: ", pop[np.argmax(fitness), :])
    pop = select(pop, fitness)
    pop_copy = pop.copy()
    
    for parent in pop:
        child = crossover(parent, pop_copy)
        child = mutate(child)
        parent[:] = child
        
print(translateDNA(pop))

[ 9.7 -3.2  9.6  9.6  9.6  9.6  9.6  9.6  9.8  9.6 -3.2  9.6  9.8  9.6
  9.6  9.6  9.7  9.7  9.6  3.2  9.6  9.6  9.6  9.6  9.6  9.6  9.6  9.6
  9.7  9.6  9.7  9.7  9.6  9.6  2.8  9.6 10.   9.2  9.6  9.6  9.6  9.6
 10.   9.6 10.   9.6  9.7  9.7  9.6  9.6  9.6  9.7  9.7 10.   3.2  3.3
  9.6 -3.2  9.7  9.6  9.6  9.6  9.6  9.6  9.6  9.6  9.6  9.6  9.2  9.6
  9.6  9.6  9.6 -1.6  9.6  3.2  9.6  9.6  9.6  9.6  9.6  2.8  9.7  9.6
  9.6  9.6  9.6  9.6 10.   9.6 10.   9.6  3.2  9.6  9.6  9.6  9.6 10.
  9.7 10. ]
